In [1]:
!pip install pyEDFlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 26.6 MB/s eta 0:00:00a 0:00:01


In [2]:
import numpy as np
from pyedflib import highlevel
import matplotlib.pyplot as plt
import torch
import torch.nn
from scipy.signal import welch
import pandas as pd

In [ ]:
sample_rate = 400

In [ ]:
file1 = "/kaggle/input/kriski/data/ECoG_fully_marked_4 2 files 6 h each/Ati4x3_12m_BL_6h_fully_marked.edf"
file2 = "/kaggle/input/kriski/data/ECoG_fully_marked_4 2 files 6 h each/Ati4x1_15m_BL_6h_fully_marked.edf"
file3 = "/kaggle/input/kriski/data/ECoG_fully_marked_4 2 files 6 h each/Ati4x6_14m_BL_6h_fully_marked.edf"

In [ ]:
signals1, _, header1 = highlevel.read_edf(file1)
time1 = np.arange(signals1.shape[1]) / sample_rate

df1 = pd.DataFrame({"time": time1, "FrL": signals1[0], "FrR": signals1[1], "OcR": signals1[2]})

In [ ]:
signals2, _, header2 = highlevel.read_edf(file2)
time2 = (np.arange(signals2.shape[1]) / sample_rate) # + (time1[-1]+time[1])

df2 = pd.DataFrame({"time": time2, "FrL": signals2[0], "FrR": signals2[1], "OcR": signals2[2]})

In [ ]:
signals3, _, header3 = highlevel.read_edf(file3)
time3 = (np.arange(signals3.shape[1]) / sample_rate) # + (time2[-1]+time[1])

df3 = pd.DataFrame({"time": time3, "FrL": signals3[0], "FrR": signals3[1], "OcR": signals3[2]})

In [ ]:
def prep_df(df, header):
    df['swd'] = 0
    df['ds'] = 0
    df['is'] = 0
    df['trash'] = 0

    def get_intervals(annotation_list, start_label, end_label):
        intervals = []
        start = None
        for time, _, label in annotation_list:
            if label == start_label:
                start = time
            elif label == end_label and start is not None:
                intervals.append((start, time))
                start = None
        return intervals

    swd_intervals = get_intervals(header['annotations'], 'swd1', 'swd2')
    ds_intervals = get_intervals(header['annotations'], 'ds1', 'ds2')
    is_intervals = get_intervals(header['annotations'], 'is1', 'is2')

    def mark_intervals(df, intervals, column_name):
        for start, end in intervals:
            df.loc[(df['time'] >= start) & (df['time'] <= end), column_name] = 1

    mark_intervals(df, swd_intervals, 'swd')
    mark_intervals(df, ds_intervals, 'ds')
    mark_intervals(df, is_intervals, 'is')

    df['trash'] = np.where((df['swd'] == 0) & (df['ds'] == 0) & (df['is'] == 0), 1, 0)
    
    return df

In [53]:
df_1 = prep_df(df1, header1)

df_2 = prep_df(df2, header2)
df_2['time'] += (list(df_1['time'])[-1] + df_1['time'][1])

df_3 = prep_df(df3, header3)
df_3['time'] += (list(df_2['time'])[-1] + df_1['time'][1])

df = pd.concat([df_1, df_2, df_3], axis=0)

In [54]:
df

,time,FrL,FrR,OcR,swd,ds,is,trash
0,0.0000,0.100812,-0.042938,0.037625,0,0,0,1
1,0.0025,0.110375,-0.089438,0.043063,0,0,0,1
2,0.0050,0.012750,-0.183125,-0.057875,0,0,0,1
3,0.0075,0.048250,-0.176437,-0.085000,0,0,0,1
4,0.0100,0.044937,-0.160938,-0.088063,0,0,0,1
...,...,...,...,...,...,...,...,...
8640395,64802.9875,0.000000,0.000000,0.000000,0,0,0,1
8640396,64802.9900,0.000000,0.000000,0.000000,0,0,0,1
8640397,64802.9925,0.000000,0.000000,0.000000,0,0,0,1
8640398,64802.9950,0.000000,0.000000,0.000000,0,0,0,1


In [55]:
df.to_csv("all.csv", index=False)